# Populate country and GDP
Pleae run kiva data population first.

This script take unique country code from Kiva data, load the country name and GDP for years after 2006 into country and country_gdp tables.

Instead of ORM, use SQL Expression Language to run queries. Also table reflection is been used. ORM is better suited to handle individual rows, and SQL Expression is better in run analytic queries and batch opertions. 


TODO: Logic: 

1) Get list of country_code from lenders
2) Get country code, name, lon/lat, and income_level from COUNTRY_URL
3) Get GDP for each year from wbdata API
 

In [2]:
import wbdata
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.sql import select
import pymysql
import requests
import json

pymysql.install_as_MySQLdb()
from config import MYSQL_URL
engine = create_engine(MYSQL_URL, encoding='utf-8')
conn = engine.connect()

metadata = MetaData()

COUNTRY_URL = 'http://api.worldbank.org/v2/countries/?format=json'

In [4]:
# Table reflection
lender = Table('lender', metadata, autoload=True, autoload_with=engine)
country = Table('country', metadata, autoload=True, autoload_with=engine)
country_gdp = Table('country_gdp', metadata, autoload=True, autoload_with=engine)


In [8]:
r = requests.get(COUNTRY_URL)

In [9]:
r.status_code

200

In [10]:
r.json()

[{'page': 1, 'pages': 7, 'per_page': '50', 'total': 304},
 [{'adminregion': {'id': '', 'iso2code': '', 'value': ''},
   'capitalCity': 'Oranjestad',
   'id': 'ABW',
   'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
   'iso2Code': 'AW',
   'latitude': '12.5167',
   'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
   'longitude': '-70.0167',
   'name': 'Aruba',
   'region': {'id': 'LCN',
    'iso2code': 'ZJ',
    'value': 'Latin America & Caribbean '}},
  {'adminregion': {'id': 'SAS', 'iso2code': '8S', 'value': 'South Asia'},
   'capitalCity': 'Kabul',
   'id': 'AFG',
   'incomeLevel': {'id': 'LIC', 'iso2code': 'XM', 'value': 'Low income'},
   'iso2Code': 'AF',
   'latitude': '34.5228',
   'lendingType': {'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'},
   'longitude': '69.1761',
   'name': 'Afghanistan',
   'region': {'id': 'SAS', 'iso2code': '8S', 'value': 'South Asia'}},
  {'adminregion': {'id': '', 'iso2code': '', 'value': ''},
   'ca

In [60]:
country_code_list = pd.read_sql(select([lender.c.country_code]).distinct(), engine)['country_code'].tolist()

In [64]:
type(country_code_list[1])

str

In [82]:
# Only load GDP data after 2006 as Kiva data started from year 2006
gdp_df = pd.DataFrame(columns=['country_code', 'year', 'gdp'])

In [95]:
for country_code in country_code_list:
    if country_code is None:
        continue
    try:
        for row in wbdata.get_data("NY.GDP.PCAP.CD", country=country_code):
            gdp_df = gdp_df.append({
                'country_code': country_code,
                'year': row['date'],
                'gdp': row['value']
            }, ignore_index=True)
    except Exception as e:
        print(e)

'NoneType' object is not iterable
'NoneType' object is not iterable
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'NoneType' object is not iterable
list index out of range
list index out of range
list index out of range
'NoneType' object is not iterable


In [99]:
gdp_df.head()

,country_code,year,gdp
0,AD,2017,None
1,AD,2016,36988.6220301891
2,AD,2015,36038.2676044596
3,AD,2014,42294.9947269717
4,AD,2013,40619.7112977795


In [102]:
gdp_df = gdp_df.dropna()

In [103]:
conn.execute('truncate country_gdp')
gdp_df.to_sql('country_gdp', engine, if_exists='append', index=False)


In [104]:
gdp_df

,country_code,year,gdp
1,AD,2016,36988.6220301891
2,AD,2015,36038.2676044596
3,AD,2014,42294.9947269717
4,AD,2013,40619.7112977795
5,AD,2012,38391.0808669785
6,AD,2011,41098.7669417228
7,AD,2010,39736.3540626699
8,AD,2009,43339.3798746543
9,AD,2008,47785.6590856793
10,AD,2007,48582.8084550866


In [79]:
wbdata.get_data("NY.GDP.PCAP.CD", country='US')

[{'country': {'id': 'US', 'value': 'United States'},
  'date': '2017',
  'decimal': '1',
  'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'value': None},
 {'country': {'id': 'US', 'value': 'United States'},
  'date': '2016',
  'decimal': '1',
  'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'value': '57638.1590879883'},
 {'country': {'id': 'US', 'value': 'United States'},
  'date': '2015',
  'decimal': '1',
  'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'value': '56469.0089691129'},
 {'country': {'id': 'US', 'value': 'United States'},
  'date': '2014',
  'decimal': '1',
  'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'value': '54706.8681977132'},
 {'country': {'id': 'US', 'value': 'United States'},
  'date': '2013',
  'decimal': '1',
  'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'value': '5278

In [10]:
gdp_df = wbdata.get_dataframe({
    "NY.GDP.PCAP.CD": 'gdp',
    },
    country=['us']
).query('date >= "2006"')
gdp_df

,gdp
date,
2017,NaN
2016,57638.159088
2015,56469.008969
2014,54706.868198
2013,52787.026949
2012,51450.122295
2011,49790.665478
2010,48373.878816
2009,47001.555350
